# Mode d'emploi de la classe CarNetwork - **Comment se rendre à l'ENSAE avec un véhicule électrique d'une autonomie de 5 kilomètres**

## Plan du notebook : 
0. Charger les packages utilisés y compris la classe CarNetwork dont on peut retrouver le code dans le fichier CarNetwork.py. 

1. Charger les données utilisées (issues du lien "https://www.data.gouv.fr/fr/datasets/r/51606633-fb13-4820-b795-9a2a575a72f1") et les nettoyer à l'aide des méthodes de la classe CarNetwork. 

2. Calcul de la distance entre les deux destinations en entrées de la classe CarNetwork. 

3. Tester la méthode .trajet_voiture() 

<font color='red'> ATTENTION : en lançant ce notebook, de nombreux fichiers de cache (data.osm et .lock) sont crées. Ceux-ci sont liés aux cartes qui sont générés dans le code. Ils expliquent les dossiers de caches qui sont associés à chaque notebook impliquant des cartes. Ils sont marqués dans le fichier .gitignore pour ne pas être pris en compte lors des commits.</font>

### 0. Chargement des packages

In [3]:
import pandas as pd

In [4]:
import pyroutelib3 
import os
import sys 

## changer le chemin selon l'utilisateur 
car_network_directory = '/Users/augustincablant/Documents/GitHub/Pycar/Modules' 

#'/home/onyxia/work/PyCar/Modules'
## si Augustin utilise le chemin absolu est '/Users/augustincablant/Documents/GitHub/Pycar/Modules' 

sys.path.append(car_network_directory)

from CarNetwork import CarNetwork

### 1. Chargement des données et nettoyage de celles-ci

**Remarques**:
- Saisir une adresse en Île-de-France, ici nous saisissons par exemple l'adresse de l'École
- Veuillez respecter le format de l'adresse
- On prend volontairement une très faible autonomie pour que des points d'arrêts s'affichent

In [5]:
## On initialise un objet de la classe CarNetwork
reseau = CarNetwork('3, rue Dareau, 75014', "5, avenue Le Chatelier Royale, 91120", 5)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

On applique la méthode .clean_data() [voir code dans la classe]

In [ ]:
# On nettoie les données à l'aide de la méthode .clean_data()
reseau.clean_data()

In [ ]:
# On affiche les données 
reseau.stations_data.head(3)

,n_amenageur,n_operateur,n_enseigne,id_station,n_station,ad_station,code_insee,xlongitude,ylatitude,nbre_pdc,...,type_prise,acces_recharge,accessibilite,observations,date_maj,source,geo_point_borne,code_insee_commune,region,departement
0,FDEL 46,BOUYGUES ENERGIES ET SERVICES,Révéo,FR*S46*P46064*001,CATUS - Quai du vert,Quai du vert 46150 CATUS,46064.0,1.335070,44.555677,8.0,...,CHADEMO,payant,24h/24 7j/7,Recharge par badge et avec une application sma...,2021-09-04,https://www.data.gouv.fr/fr/datasets/infrastru...,"44.555677,1.33507",46064,Occitanie,Lot
1,FDEL 46,BOUYGUES ENERGIES ET SERVICES,Révéo,FR*S46*P46256*002,SAINT CIRQ LAPOPIE - PARKING P4,Parking P4 46330 SAINT CIRQ LAPOPIE,46256.0,1.667674,44.463825,2.0,...,EF-T2-T3,payant,24h/24 7j/7,Recharge par badge et avec une application sma...,2021-09-04,https://www.data.gouv.fr/fr/datasets/infrastru...,"44.463825,1.667674",46256,Occitanie,Lot
2,FDEL 46,BOUYGUES ENERGIES ET SERVICES,Révéo,FR*S46*P46102*001,FIGEAC - Parking Jean Jaurès,Parking Jean Jaurès 46100 FIGEAC,46102.0,2.029835,44.606428,8.0,...,CHADEMO,payant,24h/24 7j/7,Recharge par badge et avec une application sma...,2021-09-04,https://www.data.gouv.fr/fr/datasets/infrastru...,"44.606428,2.029835",46102,Occitanie,Lot


### 2. Calcul du **trajet** et de la **distance** entre les deux localisations entrées en argument de la classe CarNetwork

Il est important de calculer cette distance. Cependant, il existe deux manières de calculer cette distance : 
- à vol d'oiseau 
- selon les routes existantes

La distance en vol d'oiseau ne nous intéresse pas, on se déplace en voiture. 

En revanche, il est important de calculer cette distance selon les routes existances car une voiture dont l'autonomie (entrée en argument de la classe CarNetwork) est supérieure à la distance entre les deux localisations suivant les routes existantes ne nécessite pas de trajet optimisé.



In [ ]:
reseau.get_coordo()

**Remarque**: la cellule suivante peut être un peu longue à charger, surtout selon l'adresse saisie 

In [ ]:
trajet = reseau.trajet_voiture()

On note que la variable trajet est une liste dont chaque élément est un tuple de longueur 2 contenant une **latitude et une longitude**. Ces localisations correspondent aux points succsessifs empruntés par l'utilisaeur lors de son trajet.

In [ ]:
## On affiche les 10 premières lignes de la liste trajet pour se faire une idée de son contenu 

trajet[:10] 

[(48.8318549, 2.3393058),
 (48.8318737, 2.3392971),
 (48.8319363, 2.3392682),
 (48.8320027, 2.3394259),
 (48.8320415, 2.3395152),
 (48.8320621, 2.3395597),
 (48.831872, 2.3402605),
 (48.8318061, 2.3405056),
 (48.8317363, 2.340765),
 (48.8317062, 2.3408756)]

On peut représenter le trajet associé à cet itinéraire sur une carte à l'aide de la méthode .get_route_map() codée dans la classe. Cette méthode repose elle-mêne sur la classe **Folium**.

In [ ]:
map1 = reseau.get_route_map()

In [ ]:
map1

On peut désormais calculer la distance associée à cet itinéraire à l'aide de la routine distance_via_routes


Cette routine renvoie deux éléments : 
1. un float *distance* qui renvoie la distance du trajet entre les points A et B.

2. une liste de coordonnées (latitude, longitude) qui correspond aux arrêts necessaires pour l'utilisateur résultant de la contrainte d'autonomie du véhicule

In [ ]:
distance, stop_coord = reseau.distance_via_routes()


### 3. Visualisation du nombre d'arrêt requis pour effectuer le trajet selon l'autonomie du véhicule

La routine .plot_stop_points() permet de visualiser les localisatins contenues dans *stop_coord* (renvoyé comme second argument de reseau).

In [ ]:
reseau.plot_stop_points(map1)

In [ ]:
map1

Sur l'exemple ci-dessus, on voit que pour se rendre à l'ENSAE avec un véhicule électrique ayant une autonomie de 5 kilomètres, il va être nécessaire de s'arrêter **4 fois**. 

Sachant que l'on va devoir s'arrêter **4 fois**, on aimerait bien représenter, pour chaque arrêt, les bornes de recharges les plus proches des points d'arrêt. 

### 4. Bornes de recharge les plus proches des points d'arrêt

L'idée est de se donner dans un premier temps une distance maximale qui correspond au rayon de recherche. Il faut que cette distance maximale soit raisonnable relativement à l'autonomie. 

Dans la suite, on prend une distance maximale de **3 kilomètres**, ce qui peut ne pas paraître raisonnable pour une autonomie de 5 kilomètres. Nous faisons ce choix pour nous assurer que des bornes seront affichées (pour un trop faible rayon, il n'y aura aucune borne à proximité).

In [ ]:
distance_max = 3
nearest_stations = reseau.nearest_stations(stop_coord, distance_max)

In [ ]:
reseau.plot_nearest_stations(map1, nearest_stations)

/home/onyxia/work/PyCar/Modules/CarNetwork.py:490: UserWarning: color argument of Icon should be one of: {'gray', 'darkpurple', 'lightred', 'beige', 'darkgreen', 'darkred', 'darkblue', 'lightgray', 'pink', 'red', 'lightblue', 'lightgreen', 'cadetblue', 'black', 'green', 'white', 'blue', 'purple', 'orange'}.
  icon=folium.Icon(color='yellow'),


**Remarque**: N'hésitez pas à bouger votre souris au dessus de chaque point représenté pour davantage d'informations !

In [ ]:
map1

Enfin, on pourrait se demander si ce trajet passe par des tronçons accidentés et, le cas échéant, en informer notre utilisateur

### 5. Visualisation des tronçons les plus accidentés en Île-de-France

On représente par un tronçon bleu large les zones les plus accidentées, on ne passe pas loin de certains axes !

In [ ]:
path = '/home/onyxia/work/PyCar/DOWNLOAD/accidents_2022_idf.xlsx'
reseau.plot_accidents(map1, path)

In [ ]:
map1

<font color='red'> ATTENTION : nous ne sommes pas parvenus à récupérer les données sur les accidents en France en 2022 sans télécharger le fichier excel (accidents_2022_idf.xlsx dans DOWNLOAD). Si un autre utilisateur souhaite lancer la cellule précédente, il doit fournir le chemin de ce fichier. C'est très peu pratique, et nous en excusons.</font>

### 6. Et si l'on souhaite visualiser toutes les bornes en Île-de-France ?